In [1]:
import requests
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from datetime import datetime as dt

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [2]:
"""
arg: None
return: driver
"""


def set_driver():
    options = Options()

    # options.set_headless(True)
    options.binary_location = "/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary"
    # options.add_argument("--headless")

    chromedriver_path = "/Users/higashi/Desktop/Document/chromedriver/chromedriver"

    return webdriver.Chrome(options=options, executable_path=chromedriver_path)


In [3]:
"""
arg: url
return: urls
"""


def get_page_links(start_url):
    urls = []
    urls.append(start_url)
    
    url = start_url
    
    for i in range(50):  # 何ページ目まで読むか
        r = requests.get(url)
        html_contents = r.text

        html_soup = BeautifulSoup(html_contents)
        try:
            next_link = 'https://weixin.sogou.com/weixin' + html_soup.find_all('a', {'id': 'sogou_next'})[0].get('href')
            if next_link in urls:
                continue
            else:
                urls.append(next_link)
                url = next_link
        except:
            break

    return urls


In [27]:
"""
arg: driver, url
return: driver, html_soup
"""


def get_page_html_soup_newdriver(driver, url):
    i = 0
    while True:
        if i == 0:
            pass
        else:
            driver = driver
        
        try:
            driver.get(url)
            html = driver.page_source.encode('utf-8')
            html_soup = BeautifulSoup(html, "html.parser")
        
            return driver, html_soup
    
        except TimeoutException:
            driver.quit()
            driver = set_driver()
            timeout = 15
            driver.set_page_load_timeout(timeout)
            # driver = login(driver, url, 'feed')  # ログイン
            i += 1


def get_page_html_soup(driver):
    html = driver.page_source.encode('utf-8')
    html_soup = BeautifulSoup(html, 'html.parser')
    
    return html_soup


In [5]:
"""
arg: html_soup
return: feed_id_list
"""


def get_feed_id_list(html_soup):
    feed_id_list = []
    div_elements = html_soup.find_all('ul', {'class': 'news-list'})
    div_elements = div_elements[0].find_all('li')
    for elem in div_elements:
        feed_id = elem.get('d')
        feed_id_list.append(feed_id)
    
    return feed_id_list


In [6]:
"""
arg: html_soup, feed_id
return: feed_link
"""


def get_feed_link(html_soup, feed_id):
    feed_ul = html_soup.find_all('ul', {'class': 'news-list'})
    feed_link = feed_ul[0].find_all('li', {'d': feed_id})[0].find_all('a')[0].get('href')
    
    return feed_link


In [7]:
"""
arg: html_soup, feed_id
return: feed_link
"""


def get_feed_account(html_soup, feed_id):
    feed_ul = html_soup.find_all('ul', {'class': 'news-list'})
    feed_account = feed_ul[0].find_all('li', {'d': feed_id})[0].find_all('a', {'class': 'account'})[0].text
    
    return feed_account


In [10]:
"""
arg: html_soup, feed_id
return: post_time
"""


def get_post_time(html_soup, feed_id):
    feed_ul = html_soup.find_all('ul', {'class': 'news-list'})
    post_time = feed_ul[0].find_all('li', {'d': feed_id})[0].find_all('span', {'class': 's2'})[0].text
    
    return post_time


In [8]:
"""
arg:html_soup
return: title
"""


def get_feed_title(html_soup):
    title = ''
    h2_tag = html_soup.find_all('h2', {'class': 'rich_media_title'})
    title_pieces = h2_tag[0].text.split()
    for title_piece in title_pieces:
        title += title_piece
    
    return title


In [9]:
"""
arg: html_soup
return: text
"""


def get_feed_text(html_soup):
    text = ''
    div_tag = html_soup.find_all('div', {'class': 'rich_media_content'})
    p_tags = div_tag[0].find_all('p')
    for p_tag in p_tags:
        text += p_tag.text
    
    return text


In [11]:
"""
STEP1
driver起動
"""

# driver初期化
timeout = 15
driver = set_driver()
driver.set_page_load_timeout(timeout)

# スタートリンクの定義
start_url = 'https://weixin.sogou.com/weixin?type=2&ie=utf8&query=%E8%B5%84%E7%94%9F%E5%A0%82%20program&tsn=5&ft=2018-12-01&et=2019-02-28&interation=&wxid=&usip='

driver.get(start_url)


In [45]:
"""
STEP2
データ定義
"""

# 出力データ
feed_id_list = []
feed_links = []
feed_titles = []
feed_texts = []
feed_accounts = []
feed_post_times = []

# 一時データ
feed_id_list_tmp = []

count = 0


In [46]:
"""
STEP3
クローリング開始
記事リンクの取得
"""


while True:
    
    # 現在のurlを取得
    current_url = driver.current_url
        
    # フィードのhtmlを取得
    page_html_soup = get_page_html_soup(driver)
    
    try:    
        if page_html_soup:
            feed_id_list_tmp.clear()
            # feed_id_listを取得
            feed_id_list_tmp = get_feed_id_list(page_html_soup)
    
            feed_id_list = feed_id_list + feed_id_list_tmp

        for i, feed_id in enumerate(feed_id_list_tmp):
            feed_link = get_feed_link(page_html_soup, feed_id)
            feed_links.append(feed_link)
            feed_account = get_feed_account(page_html_soup, feed_id)
            feed_accounts.append(feed_account)
            feed_post_time = get_post_time(page_html_soup, feed_id)
            feed_post_times.append(feed_post_time)
        
        # nextボタンをクリック
        driver.find_element_by_id("sogou_next").click()
    except:
        print(current_url)
        break


https://weixin.sogou.com/weixin?usip=&query=freeplus&ft=2018-12-01&tsn=5&et=2019-02-28&interation=&type=2&wxid=&page=39&ie=utf8


In [48]:
"""
STEP4
クローリング開始
記事テキストの取得

記事テキストの取得では、ログインは必要ない
tryが失敗した時は、driver再起動で対処可能
"""

driver = set_driver()
timeout = 15
driver.set_page_load_timeout(timeout)

j = 0
feed_titles.clear()
feed_texts.clear()

for j, feed_link in enumerate(feed_links):
    
    driver = driver
    
    try:
        driver.get(feed_link)
        driver, feed_html_soup = get_page_html_soup_newdriver(driver, feed_link)
        title = get_feed_title(feed_html_soup)
        text = get_feed_text(feed_html_soup)
        feed_titles.append(title)
        feed_texts.append(text)
        
    except:
        feed_titles.append('texts was not detected')
        feed_texts.append('texts was not detected')
        
        driver.quit()
        driver = set_driver()
        timeout = 15
        driver.set_page_load_timeout(timeout)


In [50]:
os.chdir('/Users/higashi/PycharmProjects/WebScraping/venv/wechat/data')

df = pd.DataFrame({
    'Title': feed_titles,
    'Account': feed_accounts,
    'ID': feed_id_list,
    'PostTime': feed_post_times,
    'Text': feed_texts
})

df.to_csv('20190320_wechat_freeplus_20181201-20190228_feed' + '.csv')


In [49]:
print(len(feed_links))
print(len(feed_titles))
print(len(feed_texts))
print(len(feed_accounts))
print(len(feed_id_list))
print(len(feed_post_times))

384
384
384
384
384
384


In [39]:
A = range(10)
for i in A[5:]:
    print(i)

5
6
7
8
9


In [66]:
len(feed_texts)

623

In [67]:
for i in range(10):
    try:
        a = b/0
        print(a)
    except:
        print('error')
        

error
error
error
error
error
error
error
error
error
error


In [16]:
current_url = driver.current_url

In [17]:
driver, page_html_soup = get_page_html_soup_newdriver(driver, current_url)

In [22]:
driver.get()

In [25]:
driver.find_element_by_id("sogou_next").click()

In [30]:
print(feed_post_times)

["document.write(timeConvert('1544281105'))2018-12-8", "document.write(timeConvert('1547197238'))2019-1-11", "document.write(timeConvert('1545827458'))2018-12-26", "document.write(timeConvert('1546139906'))2018-12-30", "document.write(timeConvert('1545400816'))2018-12-21", "document.write(timeConvert('1544533216'))2018-12-11", "document.write(timeConvert('1548738847'))2019-1-29", "document.write(timeConvert('1548765002'))2019-1-29", "document.write(timeConvert('1545974683'))2018-12-28", "document.write(timeConvert('1547596828'))2019-1-16", "document.write(timeConvert('1548992482'))2019-2-1", "document.write(timeConvert('1544402684'))2018-12-10", "document.write(timeConvert('1545993499'))2018-12-28", "document.write(timeConvert('1550977490'))2019-2-24", "document.write(timeConvert('1544967729'))2018-12-16", "document.write(timeConvert('1548566213'))2019-1-27", "document.write(timeConvert('1546406687'))2019-1-2", "document.write(timeConvert('1544710858'))2018-12-13", "document.write(time

In [ ]:
https://weixin.sogou.com/weixin?type=2&ie=utf8&query=freeplus&tsn=5&ft=2018-12-01&et=2019-02-28&interation=&wxid=&usip=